# Dropout

Dropout 是另一种抑制过拟合的方法，按照一定概率（dropout probability）让神经网络的一部分神经元失活（weight = 0），这就可以减轻对某些神经元的过度的依赖（权值 weight 很大），从而缓解过拟合，降低方差，达到正则化效果。由于神经网络的神经元失活是随机的，所以我们的模型也会变得多样化。

`torch.nn.Dropout(p=0.5, inplace=False)`

**参数：**
- p：被舍弃的概率，也叫失活概率

在使用 dropout 时，数据尺度会发生变化，如果设置 dropout_prob = 0.3，那么在 model.train() 训练时，数据尺度会变为原来的 70%；而在执行 model.eval() 测试时，dropout 是关闭的。**为了使数据尺度一致，在 Pytorch 中，在 model.train() 训练时，dropout 层会把所有权值乘以 1/(1-dropout_prob)**。通过这种方式，在测试时就不用对权值进行缩放，加快了测试的速度。

**PyTorch 中 Dropout 层通常放在需要 dropout 那一层的前面：**

```
class MLP(nn.Module):
    def __init__(self, neural_num, d_prob=0.5):
        super(MLP, self).__init__()
        self.linears = nn.Sequential(

            nn.Linear(1, neural_num),
            nn.ReLU(inplace=True),

            nn.Dropout(d_prob),
            nn.Linear(neural_num, neural_num),
            nn.ReLU(inplace=True),

            nn.Dropout(d_prob),
            nn.Linear(neural_num, neural_num),
            nn.ReLU(inplace=True),

            nn.Dropout(d_prob),
            nn.Linear(neural_num, 1),
        )

    def forward(self, x):
        return self.linears(x)

neural_num = 10000
x = torch.ones((neural_num, ), dtype=torch.float32)
net = MLP(neural_num, d_prob=0.5)

net.train()
y = net(x)
print("output in training mode", y)

net.eval()
y = net(x)
print("output in eval mode", y)
```